In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import _tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from clingo.control import Control

In [ ]:
def tree_to_cnf(tree, feature_names):
    tree = tree.tree_
    def traverse(node, path):
        # Check if the node is a leaf
        if tree.feature[node] == _tree.TREE_UNDEFINED:
            if tree.value[node][0, 1] > tree.value[node][0, 0]:
                return [] # Ignore Class 1 for CNF.
            else:
                return [path]
        
        # Get the split feature.
        feature = feature_names[tree.feature[node]]
        
        # Traverse left (feature == 0 for Boolean data)
        left_path = path + [f"{feature}"]
        left_cnf = traverse(tree.children_left[node], left_path)
        
        # Traverse right (feature == 1 for Boolean data)
        right_path = path + [f"-{feature}"]
        right_cnf = traverse(tree.children_right[node], right_path)
        
        return left_cnf + right_cnf

    # Start traversal from the root node
    cnf_clauses = traverse(0, [])
    
    # Combine all the clauses into a single CNF formula
    cnf_formula = " & ".join([" | ".join(clause) for clause in cnf_clauses])
    return cnf_formula

ENC = {
    "GLOBAL_CE" : "prototype/enc_def3.lp",
    "CF_CE"     : "prototype/enc_def5.lp",
    "CF_DIFF"   : "prototype/enc_def6.lp"
}

def __parse_cnf(str):
    cnf = []
    for str_cl in str.split("&"):
        cl = []
        for str_lit in str_cl.split("|"):
            s = str_lit.strip()
            if s.startswith('-'):
                cl.append((s[1:],False))
            else:
                cl.append((s,True))
        cnf.append(cl)
    
    return cnf

def __cnf_to_facts(instance_cnf,fact_cnf,foil_cnf):
    facts = []
    i = 1
    for cl in instance_cnf:
        facts.append(f"instance_cl({i}).")
        for l,t in cl:
            sign = "t" if t else "f"
            facts.append(f"lit({i},{l},{sign}).")
        i += 1
    for cl in fact_cnf:
        facts.append(f"fact_cl({i}).")
        for l,t in cl:
            sign = "t" if t else "f"
            facts.append(f"lit({i},{l},{sign}).")
        i += 1
    for cl in foil_cnf:
        facts.append(f"foil_cl({i}).")
        for l,t in cl:
            sign = "t" if t else "f"
            facts.append(f"lit({i},{l},{sign}).")
        i += 1
        
    return facts


def __cnf_to_str(cnf):
    cl_str = []
    for cl in cnf:
        parse_lit = lambda l,s: l if s else f"-{l}" 
        cl_str.append(" | ".join([ parse_lit(l,s) for l,s in cl ]))
    
    return "  &  ".join(cl_str)


def __parse_answer_set(answer_set):
    theta = {}
    theta_p = {}
    chi = {}
    for sym in answer_set.symbols(shown=True):
        if sym.match('theta_lit',3):
            c = sym.arguments[0].number
            l = str(sym.arguments[1])
            s = str(sym.arguments[2])

            if c not in theta:
                theta[c] = []
            theta[c].append((l,s == "t"))
        elif sym.match('theta_p_lit',3):
            c = sym.arguments[0].number
            l = str(sym.arguments[1])
            s = str(sym.arguments[2])

            if c not in theta_p:
                theta_p[c] = []
            theta_p[c].append((l,s == "t"))
        elif sym.match('chi_lit',3):
            c = sym.arguments[0].number
            l = str(sym.arguments[1])
            s = str(sym.arguments[2])

            if c not in chi:
                chi[c] = []
            chi[c].append((l,s == "t"))

    
    return (list(dict(sorted(theta.items())).values()), 
            list(dict(sorted(theta_p.items())).values()), 
            list(dict(sorted(chi.items())).values()))


def __clingo_solve(encoding, n_models=1, max_clauses=5, max_lits=1):
    ctl = Control(['-Wnone', '--opt-mode=optN', '-t8', f'-c n_clauses={max_clauses}' , f'-c n_lits={max_lits}', '--verbose'])
    ctl.add(encoding)
    ctl.ground([("base", [])])
    answer_set = None
    cost = None
    opt_models = 0
    def on_model(model):
        nonlocal answer_set, cost, opt_models
        prev_cost = cost
        cost = model.cost
        if prev_cost == cost:
            opt_models += 1
            theta, theta_p, chi  = __parse_answer_set(model)
            answer_set = theta, theta_p, chi
            print(f"SOLUTION (cost {cost}):")
            print("Theta:")
            print(__cnf_to_str(theta))
            print("Theta':")
            print(__cnf_to_str(theta_p))
            print("Chi:")
            print(__cnf_to_str(chi))
            print()

        if n_models > 0 and opt_models >= n_models:
            ctl.interrupt()
        
    ctl.solve(on_model=on_model)

    return answer_set

In [ ]:
# Load Breast Cancer dataset
breast_cancer = load_breast_cancer()
df = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)

# Remove spaces in feature names
df.columns = df.columns.str.replace(' ', '')

df['target'] = breast_cancer.target
X = df.drop(columns = ['target'], axis = 1)
y = df['target']

# Booleanize features using median
medians = np.median(X, axis=0)  # Compute the median for each feature
X_boolean = (X >= medians).astype(int)  # Convert each feature to 0 or 1 based on the median

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_boolean, y, test_size=0.2, random_state=42)

In [ ]:
# First model.

model1 = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 10, random_state = 42)
model1.fit(X_train, y_train)

y_pred_1 = model1.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_1)

cnf1 = tree_to_cnf(model1, X.columns)

print(f'First model as a CNF: {cnf1}')
print(f"Accuracy of the first model: {accuracy:.2f}")

# Second model.

model2 = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 100, random_state = 42)
model2.fit(X_train, y_train)

y_pred_2 = model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_2)

cnf2 = tree_to_cnf(model2, X.columns)

print(f'Second model as a CNF: {cnf2}')
print(f"Accuracy of the second model: {accuracy:.2f}")
print()

In [ ]:
# Compute disagreements between the decision trees.
disagreements = np.where(y_pred_1 != y_pred_2)[0]
for i in disagreements:
    instance = str()
    instance_as_dict = X_test.iloc[i].to_dict()
    instance_as_dict = {k : v for (k,v) in instance_as_dict.items() if k in cnf1 or k in cnf2}
    for j in range(len(instance_as_dict.keys())):
        key = list(instance_as_dict.keys())[j]
        if j > 0:
            instance += '&'
        if instance_as_dict[key] == 0:
            instance += f'-{key}'
        else:
            instance += f'{key}'
    instance = __parse_cnf(instance)
    if y_pred_1[i] > y_pred_2[i]:
        fact = __parse_cnf(cnf1)
        foil = __parse_cnf(cnf2)
    else:
        fact = __parse_cnf(cnf2)
        foil = __parse_cnf(cnf1)

    encoding = "".join(__cnf_to_facts(instance,fact,foil))
    with open(ENC['CF_DIFF'], "r") as f:
        encoding += f.read()
    __clingo_solve(encoding)